In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/planet-datasetm/test_v2_file_mapping.csv
/kaggle/input/planet-dataset-m/Kaggle-planet-train-tif.torrent
/kaggle/input/planet-dataset-m/train_v2.csv
/kaggle/input/planet-dataset-m/Kaggle-planet-test-tif.torrent
/kaggle/input/planet-dataset-m/sample_submission_v2.csv


In [2]:
!pip install pathlib

     |████████████████████████████████| 49 kB 1.8 MB/s             
  Preparing metadata (setup.py) ... - done
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14363 sha256=e28139601620762fb8c8693b5a6fd482fdcb168166effe9a996d5ccd6d81fc76
  Stored in directory: /root/.cache/pip/wheels/6e/96/b8/10037fe231e23970bac58361d7c93571ab983a7bbc55e68550
Successfully built pathlib


In [3]:
print(os.listdir("../input/planet-dataset-m"))
print(os.listdir("../input/planet-datasetm"))


['Kaggle-planet-train-tif.torrent', 'train_v2.csv', 'Kaggle-planet-test-tif.torrent', 'sample_submission_v2.csv']
['test_v2_file_mapping.csv']


In [4]:
#training data
df_train = pd.read_csv('../input/planet-dataset-m/train_v2.csv')
print('Training data shape: ', df_train.shape)
df_train.head()

Training data shape:  (40479, 2)


,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [5]:
# Testing data features
import numpy as np
import pandas as pd
df_test = pd.read_csv('../input/planet-datasetm/test_v2_file_mapping.csv')
print('Testing data shape: ', df_test.shape)
df_test.head()

Testing data shape:  (20522, 2)


,old,new
0,file_4158.tif,file_18830.tif
1,file_1668.tif,file_19164.tif
2,file_2411.tif,file_15749.tif
3,file_16047.tif,file_7531.tif
4,file_1271.tif,file_18347.tif


In [6]:
df_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40479 entries, 0 to 40478
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  40479 non-null  object
 1   tags        40479 non-null  object
dtypes: object(2)
memory usage: 632.6+ KB


In [7]:
# Number of each type of column
df_train.dtypes.value_counts()

object    2
dtype: int64

In [8]:
# Number of unique classes in each object column
df_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

image_name    40479
tags            449
dtype: int64

In [9]:
pd.set_option('display.max_rows', 500)
import gc

# sklearn preprocessing 
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

import lightgbm as lgb


#eda
!pip install klib
import klib

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


seed = 2357
np.random.seed(seed)

In [10]:
df_cleaned_train = klib.data_cleaning(df_train)
klib.missingval_plot(df_cleaned_train)

Shape of cleaned data: (40479, 2)Remaining NAs: 0

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.21 MB (-33.87%)

No missing values found in the dataset.


In [11]:
#Preprocessing data
# combine train and test together to do common feature engineering

train_copy = df_train.copy()
test_copy = df_test.copy()

# set up a flag field to distinguish records from training and testing sets in the combined dataset
train_copy['source'] = 0
test_copy['source'] = 1


all_data = pd.concat([train_copy, test_copy], axis=0, copy=True)
del train_copy
del test_copy
gc.collect()

66

In [12]:
categorical_columns = all_data.select_dtypes('object').columns
categorical_columns

Index(['image_name', 'tags', 'old', 'new'], dtype='object')

In [13]:
objList = all_data.select_dtypes(include = "object").columns
print (objList)


# Create a label encoder object
le = LabelEncoder()
for feat in objList:
    all_data[feat] = le.fit_transform(all_data[feat].astype(str))

print (all_data.info())

Index(['image_name', 'tags', 'old', 'new'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 0 to 20521
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   image_name  61001 non-null  int64
 1   tags        61001 non-null  int64
 2   source      61001 non-null  int64
 3   old         61001 non-null  int64
 4   new         61001 non-null  int64
dtypes: int64(5)
memory usage: 2.8 MB
None


In [14]:
all_data[categorical_columns].head()

,image_name,tags,old,new
0,1,422,20522,20522
1,2,153,20522,20522
2,11113,369,20522,20522
3,22224,369,20522,20522
4,33335,128,20522,20522


In [15]:
# split the all-data DF into training and testing again
training = all_data[all_data['source']==0]
testing = all_data[all_data['source']==1]

del all_data
gc.collect()

84

In [16]:
print(training.shape)
print(testing.shape)

(40479, 5)
(20522, 5)


In [17]:
#BASELINE
#Logistic Regression

TARGET = 'image_name'
train_labels = training[TARGET]
train = training.drop(columns = [TARGET,'source'])
features = list(train.columns)
test = testing.drop(columns = ['source'])
print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (40479, 3)
Testing data shape:  (20522, 4)


In [18]:
for col in training.columns:
    col_type = training[col].dtype

    if col_type != object:
        c_min = training[col].min()
        c_max = training[col].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                training[col] = training[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                training[col] = training[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                training[col] = training[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                training[col] = training[col].astype(np.int64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                training[col] = training[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                training[col] = training[col].astype(np.float32)
            else:
                training[col] = training[col].astype(np.float64)
    else:
        training[col] = training[col].astype('category')
        
start_mem = training.memory_usage().sum() / 1024**2
end_mem = training.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

return(training)
        
        

Memory usage after optimization is: 0.73 MB
Decreased by 0.0%


SyntaxError: 'return' outside function (225110008.py, line 31)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.1)

# Train on the training data
log_reg.fit(train, train_labels)

In [ ]:
# Make the random forest classifier
rf = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1)

In [ ]:
df_train.to_csv('df_train.csv', index=False)

In [ ]:
df_test.to_csv('df_test.csv', index=False)

In [ ]:
# Make the model with the specified regularization parameter
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C = 0.1)

# Train on the training data
log_reg.fit(train, train_labels)

In [ ]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
data_path = Path("'../input/planet-dataset-m/train_v2.csv")
test_base = pd.read_csv(
    data_path /''../input/planet-dataset-m/train_v2.csv'')
sub = pd.read_csv(
    data_path / '../input/planet-dataset-m/train_v2.csv')

sub.assign(
    latDeg = test_base.latDeg,
    lngDeg = test_base.lngDeg
).to_csv(
    'submission.csv', index=False
)
